In [ ]:
import pandas as pd

pd.options.plotting.backend = "plotly"
import numpy as np
import plotly.graph_objects as go


from ares_py.class_project import Project
from ares_py.tools.get_ld import get_ld

**CREATE PROJECT**

1. Run init project with project name input -> prj class, creates folder structure
1. Run prj.Process_2dm() - with **flat mode coordinates**


In [ ]:
prj = Project("kysuce", crs=8353, res_range=[1, 1000])
prj = prj.Process_2dm(crd_mode="flat")

**PROCESS - ert_plan_pt**

1. Draw planed lines - **ert.gpkg - ert_plan_ls**
1. Copy **dtm.tif** into project/qgis folder
1. Run model **ert_ls2pt** in QGIS project and save data manually to **./tmp/tmp_plan.gpkg**
1. Run prj.Process_crd_plan () -> **input/{line}_plan.csv**, **ert.gpkg - ert_plan_pt**
1. Run prj.Process_2dm() -> loads 2dm, adds coordinates from **input/{line}_plan.csv**


In [ ]:
prj = prj.Process_crd_plan()
prj = prj.Process_2dm(crd_mode="plan")

**PROCESS man coordinates**
1. Run prj.Process_crd_rec() -> **crd_man_input_pt** *(if not exist)*
1. Edit man pt coordinates **ert.gpkg - crd_man_input_pt**
1. Run model rec proc_man  and save to  **tmp/tmp_rec.gpkg**  *(interpolated (1m) coordinates from ert.gpkg - crd_man_input_pt, DTM sampled)*
1. Run prj.Process_crd_man() -> **crd_man_proc_pt** *(coordinates with recalculated ld)*, **input/{line}_man.csv**

In [ ]:
prj = prj.Process_crd_rec()
prj = prj.Process_crd_man()

**PROCESS 2dm**

1. Set coordinates mode to select coordinates type from csv file:
    + *plan - original planned coordinates*
    + *rec - recorded in field*
    + *man - manual coordinates*
1. Run prj.Process_2dm() -> loads 2dm, adds coordinates from **input/{line}_man.csv**
1. Run prj.Proc_topo_2d() -> exports topo to **ert.gpkg -ert2d_topo_pt, ert2d_topo_ls**


In [ ]:
prj = prj.Process_2dm(crd_mode="man")
prj = prj.Proc_topo_2d()

**Process inversion outputs**
1. Do the inversions - export data to **inv/** folder formats:
    + zond - _znd.dat
    + r2d - _topres.dat

1. Run prj.Export_gridding_input() - reads and formats inversion outputs -> **04_grd/{line}.csv**
2. Run prj.Export_gpkg_inv() - exports inversion points to -> **ert.gpkg - inv_{type}_pt**
    + type = r2d / znd
1. Run prj.Export_mask()->  **ert.gpkg - mask_pl**


In [ ]:
prj = prj.Export_gridding_input()

prj = prj.Export_gpkg_inv()
prj = prj.Export_mask()

**Grid data and process contours**
1. Grid grd inputs manually or run prj.Grid_data() to grid and export surfer grids 
    * enter skip list to ignore lines (eg. for long lines)
    * runs gridding on all .csv files in **04_grd/** (except skipped lines)
    * -> **04_grd/{line}_{input_name}.grd

1. Run prj.Export_contours()
    + process contours for all files in **04_grd/** folder
    + -> **ert.gpkg - grd_contours_{type}_ls**


In [ ]:
prj = prj.Grid_data(skip=[1, 2, 3, 4, 5, 6], cell_size=1)
prj = prj.Export_contours()

In [ ]:
ld = get_ld(prj.fps["grd"], ext=".grd")
ld["inv_type"] = ld["f"].str.slice(4, None)
for type in ld["inv_type"].unique():
    print(type)

**Generate atlas_qc:**
1. Enter atlas input: 
    + *res_range_qc - res_min and res_max for qc graphs in Ohmm*
    + *page heights*
    + *map scales*
    + *lvl_frequency = layer frequency for qc graphs 1= all, 2= every 2nd...*
    + *z_gap = offset from top wof atlas extent for grid maps in meters*
1. Run prj.Atlas_qc() -> **ert.gpkg - atlas_qc**
1. Run prj.QC_lines() -> **qgis/{line}.png, qgis/{line}.pgw,** 

In [ ]:
# input - dd_lavka
# atlas_input = dict(
#     res_min=[100] * 5,
#     res_max=[20000] * 5,
#     page_heights=[594, 420, 420, 420, 420],
#     map_scales=[1000] * 5,
#     lvl_frequency=5 * [3],
#     z_gap=[15, 15, 15, 15, 5],
# )

atlas_input = dict(
    res_min=[0] * 6,
    res_max=[500] * 6,
    res_max_qc_decimation=10,
    page_heights=[420] * 6,
    map_scales=[500] * 6,
    lvl_frequency=6 * [3],
    z_gap=[15] * 6,
)


prj = prj.Atlas_qc(atlas_input)
prj = prj.QC_lines()
pd.DataFrame(prj.atlas_qc)